---
##  News Craling 기사 토큰화 하기 

In [3]:
def basicInstalls():
    import subprocess,sys
    # pip가 없으면 pip를 설치한다.
    try:import pip
    except ImportError:
        print("Install pip for python3")
        subprocess.call(['sudo', 'apt-get', 'install', 'python3-pip'])
    try:import selenium        
    except ModuleNotFoundError:
        print("Install Selenium in python3")
        subprocess.call([sys.executable, "-m", "pip", "install", 'selenium'])
    finally:import selenium 

    try:import webdriver_manager
    except ModuleNotFoundError: 
        print("Install webdriver_manager")
        subprocess.call([sys.executable, "-m", "pip", "install", 'webdriver_manager'])
    finally:import webdriver_manager
# basicInstalls()    
# 크롤링 실패 
def Crawling():
    ### Description ###
    """
        NaverNews Crawling 
        Update:
        2024.06.28 by pdg : basic setting update
            * 기사 하나 긁어서 저장하기 테스트 
    """
    import time, datetime
    import pandas as pd,  numpy as np, matplotlib.pyplot as plt

    ## Beautiful Soup
    from bs4 import BeautifulSoup 

    ## Selenium 
    from selenium import webdriver
    from selenium.webdriver.chrome.service import Service
    from selenium.webdriver.support import expected_conditions as EC
    from selenium.webdriver.support.ui import WebDriverWait
    
    ## Web driver
    from webdriver_manager.chrome import ChromeDriverManager
    from selenium.webdriver.common.by import By
    ## URL
    import requests
    import urllib.request as req
    # Chrome Browser & Chrome Driver Version Check
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument('--headless')  # 헤드리스 모드 설정 : 웹브라우져가 뒤에서 실행
    driver = webdriver.Chrome(service = Service(ChromeDriverManager().install()), options = chrome_options)
    #사이트 열고 기다림 
    time.sleep(np.random.rand()+0.83)
    
    ## 특정 날짜가 없을때  현재 날짜를  세팅
    date = ""
    if date == "":
        today = datetime.date.today()
        date = today.strftime("%Y%m%d")
        print(" 현재 일자 : ",date)
    
    ### URL Setting
    link = f'https://news.naver.com/section/103/&date='
    link = link+date
    ## link 접속
    driver.get(link)
    # driver.implicitly_wait(10000)  # 10초 동안 페이지 로딩 대기
    more_button = WebDriverWait(driver, 2).until(
        EC.presence_of_element_located((By.CLASS_NAME, 'section_more_inner._CONTENT_LIST_LOAD_MORE_BUTTON'))
    )
    itter = 0
    while itter < 2 : # 더보기 5회 
        try :
            itter+=1 
            more_button.click()
            time.sleep(np.random.rand()+0.1)
        except :continue
        
    headers = {'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64)AppleWebKit/537.36 (KHTML, like Gecko) Chrome/73.0.3683.86 Safari/537.36'}

    articles = driver.find_elements(By.CLASS_NAME, 'sa_text_title')
   
    # print(soup)

    # container = soup.select('.sa_item')
    # print(len(container))
    # for i, text in enumerate(container):
    #     lis = text.find_all("li")
    #     # img = text.find_all("img")
    #     try: news_img = text.select_one("img").get("src")
    #     except: news_img = None
    
    
    
    
    print(" -----[ 뉴스 목록 ] ----")
    article_limit = 2
    for idx, article in enumerate(articles):
        if idx < article_limit:
            title = article.text.strip()
            article_link = article.get_attribute('href')
            print(f"{idx+1}.", title)
            print(f"  |- link : {article_link}")
            ##### 뉴스 기사 내용 가져오기 
            response = requests.get(article_link, headers= headers)
            if response.status_code ==200:
                ## Selenium 에서 bs4 로 넘어간다. 
                driver.execute_script("return window.location.href;")
                html = driver.page_source
                soup = BeautifulSoup(html,'html.parser')
                info = soup.find('div',{'id': 'newsct_article'}).text.strip() # 뉴스 기사 
                info = info.replace("\n","")
                print(info)

                
    
    driver.quit()
# Crawling()

### 검색된 기사 수집을 위해 crawling version 2 를 사용함. 
def Crawling_ver02(search_word ="", start_page = 1, end_page = 2):
    ### Description ###
    """
        NaverNews Crawling 
        Update:
        2024.06.28 by pdg : basic setting update
            * 기사 하나 긁어서 저장하기 테스트 
    """
    
    ## basic functions 
    import time, datetime, re , sys, random
    import pandas as pd,  numpy as np, matplotlib.pyplot as plt
    from tqdm import tqdm

    ## Beautiful Soup
    from bs4 import BeautifulSoup 

    ## Selenium 
    from selenium import webdriver
    from selenium.webdriver.chrome.service import Service
    from selenium.webdriver.support import expected_conditions as EC
    from selenium.webdriver.support.ui import WebDriverWait
    
    ## Web driver
    from webdriver_manager.chrome import ChromeDriverManager
    from selenium.webdriver.common.by import By
    ## URL
    import requests
    import urllib.request as req
    def makePgNum (num) :
        if num == 1 :
            return num
        elif num == 0 :
            return num + 1
        else :
            return num + 9 * (num - 1)
    # Crawling할 url 생성하는 함수 만들기 (검색어, Crawling 시작 Page, Crawling 종료 Page)
    def makeUrl (search, start_pg, end_pg) :
        if start_pg == end_pg :
            start_page = makePgNum(start_pg)
            url = "https://search.naver.com/search.naver?where=news&sm=tab_pge&query=" + search + "&start=" + str(start_page)
            print("생성url: ", url)
            return url
        else :
            urls = []
            for i in range(start_pg, end_pg + 1) :
                page = makePgNum(i)
                url = "https://search.naver.com/search.naver?where=news&sm=tab_pge&query=" + search + "&start=" + str(page)
                urls.append(url)
            print("생성url: ", urls)
            return urls    
    # html에서 원하는 속성 추출하는 함수 만들기 (기사, 추출하려는 속성값)
    def news_attrs_crawler (articles, attrs) :
        attrs_content = []
        for i in articles :
            attrs_content.append(i.attrs[attrs])
            #***********************************#
            time.sleep(random.uniform(0.3, 0.8))
            #***********************************#
            
        return attrs_content
    # ConnectionError방지
    headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) Chrome/98.0.4758.102"}
    # html 생성해서 기사 Crawling하는 함수 만들기(url): Link를 반환
    def articles_crawler (url) :
        # html 불러오기
        original_html = requests.get(i, headers = headers)
        #***********************************#
        time.sleep(random.uniform(0.2, 0.8))
        #***********************************#
        html = BeautifulSoup(original_html.text, "html.parser")

        url_naver = html.select("div.group_news > ul.list_news > li div.news_area > div.news_info > div.info_group > a.info")
        url = news_attrs_crawler(url_naver, 'href')
        
        return url
    # NAVER url 생성
    url = makeUrl(search_word, start_page, end_page)
    # News Crawler 실행
    news_titles = []
    news_url = []
    news_contents = []
    news_dates = []
    for i in url :
        url = articles_crawler(url)
        news_url.append(url)
    # 제목, Link, 내용 1차원 List로 꺼내는 함수 생성
    def makeList (newlist, content) :
        for i in content :
            for j in i :
                newlist.append(j)
        return newlist
    # 제목, Link, 내용 담을 List 생성
    news_url_1 = []
    # 1차원 List로 만들기 (내용 제외)
    makeList(news_url_1, news_url)
    # NAVER 뉴스만 남기기
    final_urls = []
    for i in tqdm(range(len(news_url_1))) :
        if "news.naver.com" in news_url_1[i] :
            final_urls.append(news_url_1[i])
        else :pass

    ##### News 내용 Crawling #####
    for i in tqdm(final_urls) :
        #각 기사 html get하기
        news = requests.get(i, headers=headers)
        #***********************************#
        time.sleep(random.uniform(0.1, 0.53))
        #***********************************#
        news_html = BeautifulSoup(news.text, "html.parser")


        # News 제목 가져오기
        title = news_html.select_one("#ct > div.media_end_head.go_trans > div.media_end_head_title > h2")
        if title == None :
            title = news_html.select_one("#content > div.end_ct > div > h2")
        # News 본문 가져오기
        content = news_html.select("article#dic_area")
        if content == [] :
            content = news_html.select("#articeBody")
        # 기사 Text만 가져오기
        # List 합치기
        content = ''.join(str(content))
        # html Tag 제거 및 Text 다듬기
        pattern1 = '<[^>]*>'
        title = re.sub(pattern = pattern1, repl = '', string = str(title))
        content = re.sub(pattern = pattern1, repl = '', string = content)
        pattern2 = """[\n\n\n\n\n// flash 오류를 우회하기 위한 함수 추가\nfunction _flash_removeCallback() {}"""
        content = content.replace(pattern2, '')
        news_titles.append(title)
        news_contents.append(content)
        try :
            html_date = news_html.select_one("div#ct> div.media_end_head.go_trans > div.media_end_head_info.nv_notrans > div.media_end_head_info_datestamp > div > span")
            news_date = html_date.attrs['data-date-time']
        except AttributeError :
            news_date = news_html.select_one("#content > div.end_ct > div > div.article_info > span > em")
            news_date = re.sub(pattern = pattern1, repl = '', string = str(news_date))
        # 날짜 가져오기
        news_dates.append(news_date)

    print("검색된 기사 갯수: 총 ", (end_page + 1 - start_page) * 10, '개')
    print("\n[뉴스 제목]")
    print(news_titles)
    print("\n[뉴스 링크]")
    print(final_urls)
    print("\n[뉴스 내용]")
    print(news_contents)
    print()
    print('news_title: ', len(news_titles))
    print('news_url: ', len(final_urls))
    print('news_contents: ', len(news_contents))
    print('news_dates: ', len(news_dates))
    
    
    ##### DataFrame으로 만들기 #####
    import pandas as pd

    # DataFrame 만들기
    news_df = pd.DataFrame(
    {
        'date' : news_dates,
        'title' : news_titles,
        'link' : final_urls,
        'content' : news_contents,
    }
    )

    print(news_df)

    # 중복 Row(행) 지우기
    news_df = news_df.drop_duplicates(keep = 'first', ignore_index = True)
    print("중복 제거 후 행 개수: ", len(news_df))

    # Save to DataFrame
    now = datetime.datetime.now() 
    news_df.to_csv("./NewsData/{}_{}.csv".format(search_word, now.strftime('%Y%m%d_%H시%M분')), encoding = 'utf-8-sig', index=False)
# Crawling_ver02(search_word="반도체")

---
### 기사 가져오기 

In [4]:
import pandas as pd
df = pd.read_csv("./NewsData/반도체_20240628_17시32분.csv")


In [5]:
df.columns
test = df['content'][0]

In [6]:
import transformers
from transformers import pipeline 
classifier = pipeline("sentiment-analysis") ## text-classification 에러 남

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
/opt/anaconda3/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [7]:

def blue(str):
    from Functions import Service
    return Service.colored_text(str,'blue')

def yellow(str):
    from Functions import Service
    return Service.colored_text(str,'yellow')
def red(str):
    from Functions import Service
    return Service.colored_text(str,'red')

## 기사 하나를 토큰화 하여 학습

In [8]:
def characterTokenizer(test):
    ## 문자 토큰화 
    text = test
    tokenized_text = list(text)
    print(blue("0. token 화 한 텍스트:"),tokenized_text)
    print("  |- 배열 길이 : ",len(tokenized_text))
    token2idx = {character:vector for vector, character  in enumerate(sorted(set(tokenized_text)))}
    print(red("  |-token vector dict : "),token2idx) ## character -> index 매기고 수치화 함. 
    input_ids = [token2idx[token] for token in tokenized_text]
    print(yellow("1. Vector 화 한 기사 : "), input_ids)
    ## one hot encoding 함. 

    import torch 
    import torch.nn.functional as F
    import warnings ; warnings.filterwarnings("ignore")

    input_ids = torch.tensor(input_ids)
    print(yellow("2. input_ids(tensor) : "),input_ids)
    one_hot_encodings = F.one_hot(input_ids, num_classes=len(token2idx))
    print(yellow(f"3. one hot size : {one_hot_encodings.shape}"))
    print(one_hot_encodings)
    print(yellow(f" 첫번째 토큰의 원핫 인코딩 값: {one_hot_encodings[0]}"))
characterTokenizer(test)

0. token 화 한 텍스트: ['[', '\n', '삼', '성', '·', 'S', 'K', '·', '인', '텔', '·', 'I', 'B', 'M', ' ', '등', ' ', '참', '여', '…', '안', '덕', '근', ' ', '산', '업', '장', '관', ' ', '"', '양', '국', ' ', '기', '술', '협', '력', ' ', '지', '원', '"', '美', '상', '무', '장', '관', ' ', '"', 'A', 'I', ' ', '때', '문', '에', ' ', '반', '도', '체', ' ', '더', ' ', '중', '요', '해', '져', '…', '미', '국', ' ', '생', '산', ' ', '필', '요', '하', '다', '"', '\n', '\n', '\n', '\n', '한', '미', ' ', '공', '급', '망', '·', '산', '업', ' ', '대', '화', '(', 'S', 'C', 'C', 'D', ')', ' ', '반', '도', '체', ' ', '포', '럼', '(', '워', '싱', '턴', '=', '연', '합', '뉴', '스', ')', ' ', '김', '동', '현', ' ', '특', '파', '원', ' ', '=', ' ', '한', '국', '반', '도', '체', '산', '업', '협', '회', ' ', '김', '정', '회', ' ', '상', '근', '부', '회', '장', '과', ' ', '존', ' ', '뉴', '퍼', ' ', '미', '국', '반', '도', '체', '산', '업', '협', '회', ' ', '회', '장', '이', ' ', '2', '7', '일', '(', '현', '지', '시', '간', ')', ' ', '워', '싱', '턴', 'D', 'C', '의', ' ', '한', ' ', '호', '텔', '에', '서', ' ', '열', '린', ' ', "'", '

In [9]:
## 단어 토큰화 

test.split()
#WordPiece 사용 
from transformers import AutoTokenizer
model_checkpoint = 'distilbert-base-uncased'
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)



In [10]:
encoded_text = tokenizer(test)

Token indices sequence length is longer than the specified maximum sequence length for this model (2431 > 512). Running this sequence through the model will result in indexing errors


### 영문은 되지만 한글은 토큰화가 안된다!! 

In [31]:
def NLPInstalls():
    import subprocess,sys
    # pip가 없으면 pip를 설치한다.
    try:import pip
    except ImportError:
        print("Install pip for python3")
        subprocess.call(['sudo', 'apt-get', 'install', 'python3-pip'])
    try:import tweepy        
    except ModuleNotFoundError:
        print("Install Selenium in python3")
        subprocess.call([sys.executable, "-m", "pip", "install", 'tweepy==3.10.0'])
    finally:import tweepy 

    try:import konlpy
    except ModuleNotFoundError: 
        print("Install webdriver_manager")
        subprocess.call([sys.executable, "-m", "pip", "install", 'konlpy'])
    finally:import konlpy

from konlpy.tag import Okt,Kkma,Hannanum
tokenizer_kor = Okt()

# 형태소 분리 
poss_dicet = {}
from tqdm import tqdm 
poss = tokenizer_kor.pos(test, norm=True, stem=True)
word_list =[]
for word, tag in poss:
    if tag in ['Noun', 'Adjective','Verb','Adverb']:
        word_list.append(word)
# 분리 된 배열 
tokens = word_list
print(tokens)
word_to_index = {word:index for index, word in enumerate(sorted(set(tokens)))}
print(yellow(f"내장 함수 단어 집합({len(word_to_index.keys())}): "), word_to_index)
# 블로그 참고 :https://velog.io/@kh7ae/NLP-One-Hot-Encoding
## keras 를 이용한 one hot encoding 
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
tokenizer_keras = Tokenizer()
tokenizer_keras.fit_on_texts(word_list)
# encoded_text = tokenizer_keras.text
print(yellow(f"keras 단어집합:({len(tokenizer_keras.word_index.keys())}) "),tokenizer_keras.word_index)
encoded = tokenizer_keras.texts_to_sequences([test])[0]
print(red("뉴스본문 인코딩:"),encoded)
one_hot = to_categorical(encoded)
print(red("One hot vector result! \n"), one_hot)



['삼성', '인텔', '등', '참여', '안덕', '산업', '장관', '양국', '기술', '협력', '지원', '상무', '장관', '때문', '반도체', '더', '중요하다', '미국', '생산', '필요하다', '한미', '공급망', '산업', '대화', '반도체', '포럼', '워싱턴', '연합뉴스', '김동현', '특파원', '한국', '반도체', '산업', '협회', '김정회', '상근', '부회장', '존', '뉴퍼', '미국', '반도체', '산업', '협회', '회장', '현지', '시간', '워싱턴', '의', '하다', '호텔', '열리다', '한미', '공급망', '산업', '대화', '반도체', '포럼', '반도체', '공급망', '협력', '위', '양해각서', '를', '체결', '있다', '안덕', '산업', '통상', '자원부', '장관', '지나', '러몬', '미국', '상무부', '장관', '서명', '지켜보다', '있다', '세종', '워싱턴', '연합뉴스', '차', '대운', '기자', '김동현', '특파원', '한미', '양국', '반도체', '산업', '분야', '협력', '강화하다', '위', '한미', '공급망', '산업', '대화', '반도체', '포럼', '이', '현지', '시간', '미국', '워싱턴', '개최', '돼다', '한미', '산업', '장관', '주관', '하다', '첨단', '산업', '및', '공급망', '분야', '정례', '협의', '채널', '개최', '계기', '양국', '반도체', '협회', '주최', '처음', '열리다', '민관', '포럼', '안덕', '산업부', '장관', '지나', '러몬', '미국', '상무부', '장관', '등', '양국', '정부', '인사', '삼성', '전자', '하이닉스', '인텔', '시놉시스', '등', '양국', '반도체', '기업', '관계자', '참석', '하다', '참석자', '양국', '반도체', '산업', '현안', '기술',

In [35]:
from transformers import AutoTokenizer
from transformers import DistilBertTokenizer

model_ckpt = 'distilbert-base-uncased' ## chkpt : check point
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

## 토크나이저를 통해서 토큰화 작업.
tokens = tokenizer.convert_ids_to_tokens(encoded.input_ids)
encoded_text

AttributeError: 'list' object has no attribute 'input_ids'

==> Downloading https://formulae.brew.sh/api/formula.jws.json

==> Downloading https://formulae.brew.sh/api/cask.jws.json

==> Downloading https://ghcr.io/v2/homebrew/core/autoconf/manifests/2.72
######################################################################### 100.0%
==> Fetching dependencies for autoconf: m4
==> Downloading https://ghcr.io/v2/homebrew/core/m4/manifests/1.4.19
######################################################################### 100.0%
==> Fetching m4
==> Downloading https://ghcr.io/v2/homebrew/core/m4/blobs/sha256:f42d89db519a07d
######################################################################### 100.0%
==> Fetching autoconf
==> Downloading https://ghcr.io/v2/homebrew/core/autoconf/blobs/sha256:bb39057e8
######################################################################### 100.0%
==> Downloading https://ghcr.io/v2/homebrew/core/automake/manifests/1.16.5
######################################################################### 100.0%
==> Fetching